# Exercício de Integração ETL, ELT, APIs e Arquivos


## Trabalho em Grupo

Você trabalha em um e-commerce de variados produtos, e no último mês os gerentes operacionais da companhia notaram que
houve uma variação muito grande em seus estoques por períodos de até 24h, cujos produtos, voltavam ao estoque
no dia seguinte o que não caracterizava uma venda concretizada.
Sabendo disso um dos diretores sugeriu um estudo sobre os carrinhos em aberto, e qual o perfil dos usuários que deixavam seus carrinhos em aberto, e para isso fora criada uma demanda para seu time.

Para resolver esta demanda, você e seu time deverão realizar a ingestão dos dados de:

- Usuários
- Carrinhos

Estes dados poderão ser obtidos através dos seguintes endpoints:

- Usuários: GET https://dummyjson.com/auth/users
- Carrinhos: GET https://dummyjson.com/auth/carts

Lembrando que estes são endpoints seguros então você deverá realizar a autenticação através do seguinte endpoint: https://dummyjson.com/auth/login

Lembre-se de verificar a documentação para obter mais detalhes do funcionamento dos endpoints https://dummyjson.com/docs/auth

Após isso, os passos de tratamento do dado, serão:

1. Após consumidos os dados, você deverá persistí-los utilizando a técnica aprendida em aula em uma pastas chamada raw no formato json

2. Consumir sua camada raw e criar uma nova entidade que relacione os clientes aos carrinhos através do atributo userId do seu carrinho e o atribute id da tabela users, feito isso gravar o resultado dessa agregação em uma pasta chamda kitchen

3. Por fim você deverá contemplar se os dados gerados são capazes de responder as seguintes perguntas:

   - Qual o estado de residencia possui clientes com mais carrinhos em aberto ?
   - Qual a idade dos usuários com mais carrinhos em aberto ?
   - Quais os produtos que mais aparecem em carrinhos em aberto ?

OBS: **Não se preocupe em fazer uma análise aprofundade, o intuito desta atividade é exercitar a construção de um fluxo de ETL ou ELT utilizando os conceitos aprendidos em sala de aula, então concentrem seus esforços no fluxo de dados**


In [ ]:
%pip install requests

In [ ]:
import requests
import json


class StoreAPIClient():

    def __init__(self, username, password):
        self.username = username
        self.password = password

    def __authentica(self):
        request_body = {
            "username": self.username,
            "password": self.password,
            "expiresInMins": 1
        }

        response = requests.post(
            url="https://dummyjson.com/auth/login",
            headers={"Content-Type": "application/json"},
            data=json.dumps(request_body)
        )

        return response.json()["token"]

    def __get_headers(self):
        access_token = self.__authentica()

        return {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }

    def get_users(self):
        response = requests.get(
            url="https://dummyjson.com/auth/users",
            headers=self.__get_headers()
        )

        return response.json()

    def get_carts(self):
        response = requests.get(
            url="https://dummyjson.com/auth/carts",
            headers=self.__get_headers()
        )

        return response.json()

In [ ]:
import pandas as pd
import json


class Pipeline():

    def __init__(self):
        self.__raw_path = "../../data/AULA03/output/raw"
        self.__kitchen_path = "../../data/AULA03/output/kitchen"
        self.__store_api_client = StoreAPIClient(
            username="kminchelle", password="0lelplR")

    def __extract_and_load(self, data: dict, target_path: str, target_format: str):
        print(
            f"Pipeline.__extract_and_load target_path={target_path} with format={target_format}")

        data_str: str = json.dumps(data)

        df_data = pd.read_json(data_str)

        self.__load(
            df=df_data,
            path=target_path,
            format=target_format
        )

        return df_data

    def __load(self, df: pd.DataFrame, format: str, path: str):
        normalized_format = format.upper()

        if normalized_format == "JSON":
            df.to_json(path, orient="records", mode="w")
        elif normalized_format == "PARQUET":
            df.to_parquet(path, index=False, compression="gzip")

    def __transform(self, df_carts: pd.DataFrame, df_users: pd.DataFrame, target_path: str, target_format: str):
        print("Pipeline.__transform")

        df_carts["idCart"] = df_carts["id"]
        df_result = df_carts.merge(
            df_users, left_on="userId", right_on="id", how="inner")

        df_result_transformed = df_result[[
            "userId", "address", "age", "products", "idCart"]]

        # Extract State from Users
        df_result_transformed["state"] = df_result_transformed["address"].apply(
            lambda obj: obj["state"]).reset_index().address

        # Extract Products From Carts
        df_result_transformed = df_result_transformed.explode("products")
        df_result_transformed["product_name"] = df_result_transformed["products"].apply(
            lambda obj: obj["title"])

        df_result_transformed = df_result_transformed[[
            "userId", "age", "state", "idCart", "product_name"]]

        self.__load(
            df=df_result_transformed,
            format=target_format,
            path=target_path
        )

    def run(self):
        users = self.__store_api_client.get_users()["users"]
        carts = self.__store_api_client.get_carts()["carts"]

        # Extract Users
        df_users = self.__extract_and_load(
            data=users,
            target_path=f"{self.__raw_path}/users.json",
            target_format="json"
        )

        # Extract Carts
        df_carts = self.__extract_and_load(
            data=carts,
            target_path=f"{self.__raw_path}/carts.json",
            target_format="json"
        )

        # Apply Data Transformations
        self.__transform(
            df_carts=df_carts,
            df_users=df_users,
            target_path=f"{self.__kitchen_path}/products_by_customer.parquet.gz",
            target_format="parquet"
        )

In [ ]:
# Executando Nosso Pipeline
Pipeline().run()

In [ ]:
import pandas as pd

products_by_customer = pd.read_parquet(
    "../../data/AULA03/output/kitchen/products_by_customer.parquet.gz")

df_states = products_by_customer.groupby(
    ["state"])["idCart"].count().reset_index(name="carts")

display(df_states)

In [ ]:
import pandas as pd

products_by_customer = pd.read_parquet(
    "../../data/AULA03/output/kitchen/products_by_customer.parquet.gz")

df_ages = products_by_customer.groupby(
    ["age"])["idCart"].count().reset_index(name="carts")

display(df_ages)

In [ ]:
import pandas as pd

products_by_customer = pd.read_parquet(
    "../../data/AULA03/output/kitchen/products_by_customer.parquet.gz")

df_products = products_by_customer.groupby(
    ["product_name"])["product_name"].count().reset_index(name="occurences")

display(df_products)